# EDA Limpio (sin errores Ruff)

In [ ]:
import matplotlib.pyplot as plt

# Cargar dataset (ajusta la ruta manualmente si lo usas en tu flujo)
# df = pd.read_csv("../../data/raw/data.csv")


In [ ]:

def bar_by_category(df, value_col, cat_col, agg="mean"):
    """Genera un gráfico de barras agrupando value_col por cat_col."""
    plt.figure(figsize=(8, 4))
    tabla = df.groupby(cat_col)[value_col].agg(agg)
    tabla.plot(kind="bar", title=f"{value_col} por {cat_col} ({agg})")
    plt.ylabel(value_col)
    plt.grid(True, axis="y")
    plt.tight_layout()
    plt.show()
    return tabla


In [ ]:

# Ejemplo de uso si el dataframe está cargado en df
# if "Usage_kWh" in df.columns:
#     for cat in ["WeekStatus", "Day_of_week", "Load_Type"]:
#         if cat in df.columns:
#             tabla = bar_by_category(df, "Usage_kWh", cat)
#             display(tabla)
